In [23]:
import numpy as np 
import pandas as pd 
from datetime import date
from pandas_datareader import data
import requests 
import json
from dateutil.relativedelta import relativedelta


In [24]:
ratios = [0,0.236, 0.382, 0.5 , 0.618, 0.786,1,1.414,1.618]
fiblevels = np.array(ratios)

In [25]:
def RSI(series, period=14):
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    return 100 - 100 / (1 + rs)


# calculating Stoch RSI (gives the same values as TradingView)
# https://www.tradingview.com/wiki/Stochastic_RSI_(STOCH_RSI) 
def STRSIA(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.rolling(smoothK).mean()
    stochrsi_D = stochrsi_K.rolling(smoothD).mean()
    return stochrsi, stochrsi_K, stochrsi_D

In [26]:
ratios

[0, 0.236, 0.382, 0.5, 0.618, 0.786, 1, 1.414, 1.618]

In [27]:
## get the data 
import os
symbols= ['OMGUSDT']
live = pd.to_datetime('today')
start = pd.to_datetime('2020-01-01')
end = pd.to_datetime('2021-10-31')

root_url = 'https://api.binance.com/api/v1/klines'
symbol = 'ADA'
interval = '1h'
url = root_url + '?symbol=' + symbol + '&interval=' + interval
print(url)


directory = 'BN-main'
table = []
   
        
        
        #now = df.close[df.close.idxmax()]
#get the live price 
        #array = filename.split("/")
        #symbol =array[0]+'USDT'
for symbol in symbols:
        url = root_url + '?symbol=' + symbol + '&interval=' + interval
        data = json.loads(requests.get(url).text)
        dflive = pd.DataFrame(data ,columns =['open_time',
              'o', 'h', 'l', 'c', 'v',
              'close_time', 'qav', 'num_trades',
              'taker_base_vol', 'taker_quote_vol', 'ignore'])
        dflive.open_time = pd.to_datetime(dflive['open_time'],unit='ms')
        dflive=dflive.loc[(dflive.open_time>start)]

dflive


https://api.binance.com/api/v1/klines?symbol=ADA&interval=1h


,open_time,o,h,l,c,v,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
0,2021-10-22 12:00:00,14.96900000,14.98600000,14.84000000,14.97700000,247200.90000000,1634907599999,3683895.96070000,6771,129944.20000000,1936429.45970000,0
1,2021-10-22 13:00:00,14.97800000,14.97800000,14.63900000,14.66900000,366619.30000000,1634911199999,5428655.91400000,12394,140140.00000000,2075624.60750000,0
2,2021-10-22 14:00:00,14.67000000,14.67000000,14.20300000,14.32300000,1165891.30000000,1634914799999,16769494.26060000,32979,513772.90000000,7390702.66310000,0
3,2021-10-22 15:00:00,14.32200000,14.38500000,14.10700000,14.23000000,629394.10000000,1634918399999,8969627.49880000,20061,299269.00000000,4265735.43160000,0
4,2021-10-22 16:00:00,14.22900000,14.30500000,14.03600000,14.24600000,609145.00000000,1634921999999,8652342.88220000,19437,288441.70000000,4097180.82650000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,2021-11-12 03:00:00,12.81600000,13.14900000,12.56300000,13.03700000,3453762.10000000,1636689599999,44622740.05310000,119902,1555384.20000000,20109505.35940000,0
496,2021-11-12 04:00:00,13.03700000,13.26000000,12.95200000,13.10600000,2753348.00000000,1636693199999,36167470.05060000,81534,1417473.10000000,18617443.50420000,0
497,2021-11-12 05:00:00,13.10600000,13.16200000,12.84700000,13.05600000,2916487.20000000,1636696799999,37959628.97470000,76562,1390706.50000000,18097001.23360000,0
498,2021-11-12 06:00:00,13.05600000,13.09700000,12.59400000,12.76600000,3578454.50000000,1636700399999,46358464.41120000,105500,1749350.30000000,22669764.95870000,0


In [28]:
ada15 = dflive[['open_time','o','h','l','c','v']]
ada15

,open_time,o,h,l,c,v
0,2021-10-22 12:00:00,14.96900000,14.98600000,14.84000000,14.97700000,247200.90000000
1,2021-10-22 13:00:00,14.97800000,14.97800000,14.63900000,14.66900000,366619.30000000
2,2021-10-22 14:00:00,14.67000000,14.67000000,14.20300000,14.32300000,1165891.30000000
3,2021-10-22 15:00:00,14.32200000,14.38500000,14.10700000,14.23000000,629394.10000000
4,2021-10-22 16:00:00,14.22900000,14.30500000,14.03600000,14.24600000,609145.00000000
...,...,...,...,...,...,...
495,2021-11-12 03:00:00,12.81600000,13.14900000,12.56300000,13.03700000,3453762.10000000
496,2021-11-12 04:00:00,13.03700000,13.26000000,12.95200000,13.10600000,2753348.00000000
497,2021-11-12 05:00:00,13.10600000,13.16200000,12.84700000,13.05600000,2916487.20000000
498,2021-11-12 06:00:00,13.05600000,13.09700000,12.59400000,12.76600000,3578454.50000000


In [29]:
ada15.columns=['date','open','high','low','close','volume']
ada15

,date,open,high,low,close,volume
0,2021-10-22 12:00:00,14.96900000,14.98600000,14.84000000,14.97700000,247200.90000000
1,2021-10-22 13:00:00,14.97800000,14.97800000,14.63900000,14.66900000,366619.30000000
2,2021-10-22 14:00:00,14.67000000,14.67000000,14.20300000,14.32300000,1165891.30000000
3,2021-10-22 15:00:00,14.32200000,14.38500000,14.10700000,14.23000000,629394.10000000
4,2021-10-22 16:00:00,14.22900000,14.30500000,14.03600000,14.24600000,609145.00000000
...,...,...,...,...,...,...
495,2021-11-12 03:00:00,12.81600000,13.14900000,12.56300000,13.03700000,3453762.10000000
496,2021-11-12 04:00:00,13.03700000,13.26000000,12.95200000,13.10600000,2753348.00000000
497,2021-11-12 05:00:00,13.10600000,13.16200000,12.84700000,13.05600000,2916487.20000000
498,2021-11-12 06:00:00,13.05600000,13.09700000,12.59400000,12.76600000,3578454.50000000


In [30]:
start = pd.to_datetime('2021-01-01')
ada15.date=pd.to_datetime(ada15['date'],unit='ms')
ada15= ada15.astype ({'open': float ,'high':float,'low':float ,'close':float ,'volume':float})


/Users/hjabbour/opt/anaconda3/envs/ML/lib/python3.7/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
start = pd.to_datetime('2021-01-01')
end = pd.to_datetime('2021-10-31')


ada2021 = ada15.loc[(ada15['date']> start)& (ada15['date']< end)].copy()
ada2021['RSI'] = RSI(ada2021['close'])
ada2021['RSI4']=RSI(ada2021['close'],56)
ada2021['RSI8']=RSI(ada2021['close'],112)
ada2021['STRSI'] = STRSIA(ada2021['close'])[0]
ada2021['STRSIK'] = STRSIA(ada2021['close'])[1]
ada2021['STRSID'] = STRSIA(ada2021['close'])[2]

ada2021['STRSI4'] = STRSIA(ada2021['close'],56,12,12)[0]
ada2021['STRSIK4'] = STRSIA(ada2021['close'],56,12,12)[1]
ada2021['STRSID4'] = STRSIA(ada2021['close'],56,12,12)[2]

ada2021['STRSI8'] = STRSIA(ada2021['close'],112,24,24)[0]
ada2021['STRSIK8'] = STRSIA(ada2021['close'],112,24,24)[1]
ada2021['STRSID8'] = STRSIA(ada2021['close'],112,24,24)[2]



print(ada2021.iloc[13:70,0:18])

                  date    open    high     low   close     volume        RSI  \
13 2021-10-23 01:00:00  14.144  14.185  14.062  14.139   136083.8        NaN   
14 2021-10-23 02:00:00  14.138  14.178  14.120  14.163    88698.2  21.736111   
15 2021-10-23 03:00:00  14.168  14.200  14.128  14.136   132925.1  21.305896   
16 2021-10-23 04:00:00  14.137  14.245  14.121  14.213   138927.8  25.815405   
17 2021-10-23 05:00:00  14.213  14.229  14.154  14.173   117219.8  25.013513   
18 2021-10-23 06:00:00  14.173  14.228  14.145  14.210   124562.2  27.264182   
19 2021-10-23 07:00:00  14.211  14.327  14.175  14.204   276596.6  27.122020   
20 2021-10-23 08:00:00  14.205  14.314  14.200  14.310   142286.2  33.699346   
21 2021-10-23 09:00:00  14.310  14.329  14.071  14.214   234647.6  30.972969   
22 2021-10-23 10:00:00  14.214  14.300  14.079  14.097   305136.0  27.999805   
23 2021-10-23 11:00:00  14.098  14.349  14.000  14.292   392818.0  38.581778   
24 2021-10-23 12:00:00  14.295  14.380  

In [32]:
#maxlevels = [13,13,13,13,13,13,13]
#maxlevels = np.array(maxlevels)
#maxlevels

In [33]:
#minlevels = [11,11,11,11,11,11,11]
#minlevels = np.array(minlevels)
#minlevels

In [34]:
ada2021

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,STRSIK,STRSID,STRSI4,STRSIK4,STRSID4,STRSI8,STRSIK8,STRSID8
0,2021-10-22 12:00:00,14.969,14.986,14.840,14.977,247200.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-10-22 13:00:00,14.978,14.978,14.639,14.669,366619.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-10-22 14:00:00,14.670,14.670,14.203,14.323,1165891.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-10-22 15:00:00,14.322,14.385,14.107,14.230,629394.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-10-22 16:00:00,14.229,14.305,14.036,14.246,609145.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,2021-10-30 19:00:00,13.761,13.765,13.686,13.725,120067.7,56.798462,51.351847,48.381782,0.319816,0.342092,0.338381,0.790049,0.858507,0.874783,NaN,NaN,NaN
200,2021-10-30 20:00:00,13.725,13.729,13.598,13.622,114787.9,52.429232,50.356345,47.888684,0.000000,0.241125,0.307272,0.718272,0.835030,0.872108,NaN,NaN,NaN
201,2021-10-30 21:00:00,13.619,13.647,13.575,13.633,111160.1,52.846412,50.460773,47.945853,0.020163,0.113326,0.232181,0.725801,0.812180,0.867014,NaN,NaN,NaN
202,2021-10-30 22:00:00,13.632,13.656,13.307,13.542,289927.1,49.016746,49.582250,47.510781,0.000000,0.006721,0.120391,0.662458,0.784051,0.858933,NaN,NaN,NaN


In [35]:
high = ada2021.high
low = ada2021.low
maximums5 = high.rolling(5,5)
max5 = maximums5.max()
minimums5 = low.rolling(5,5)
min5 = minimums5.min()
maximums10 = high.rolling(10,10)
max10 = maximums10.max()
minimums10 = low.rolling(10,10)
min10 = minimums10.min()


In [36]:
len(max5)

204

In [37]:

ada2021['max5'] = max5
ada2021['max10'] = max10
ada2021['min5'] = min5
ada2021['min10'] = min10

ada2021.tail()

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,...,STRSI4,STRSIK4,STRSID4,STRSI8,STRSIK8,STRSID8,max5,max10,min5,min10
199,2021-10-30 19:00:00,13.761,13.765,13.686,13.725,120067.7,56.798462,51.351847,48.381782,0.319816,...,0.790049,0.858507,0.874783,NaN,NaN,NaN,13.779,14.191,13.494,13.414
200,2021-10-30 20:00:00,13.725,13.729,13.598,13.622,114787.9,52.429232,50.356345,47.888684,0.000000,...,0.718272,0.835030,0.872108,NaN,NaN,NaN,13.779,14.191,13.546,13.414
201,2021-10-30 21:00:00,13.619,13.647,13.575,13.633,111160.1,52.846412,50.460773,47.945853,0.020163,...,0.725801,0.812180,0.867014,NaN,NaN,NaN,13.779,14.054,13.575,13.414
202,2021-10-30 22:00:00,13.632,13.656,13.307,13.542,289927.1,49.016746,49.582250,47.510781,0.000000,...,0.662458,0.784051,0.858933,NaN,NaN,NaN,13.779,14.054,13.307,13.307
203,2021-10-30 23:00:00,13.541,13.557,13.339,13.540,292757.3,48.932816,49.562941,47.501223,0.000000,...,0.661066,0.762110,0.849535,NaN,NaN,NaN,13.765,13.869,13.307,13.307


In [38]:
#maxnew = np.ones_like(ratios)
#maxnew

In [39]:
#addmin = mintest.tolist()
#addmax = maxtest.tolist()
#data = pd.DataFrame({"max":maxtest,"min":mintest},dtype='float')
#data = data.append(addmin,ignore_index=True)
#data1 = pd.DataFrame(maxtest)
#data2 = pd.DataFrame(mintest)
#data = pd.DataFrame(data1,data2)

In [40]:
#datamul = data.mul([2,1],axis='columns')
#datamul = data['max'].mul(ratios[7])

In [41]:
#datamul

In [42]:
#df = ada2021.loc[(ada2021['max10']>0)& (ada2021['min10']>0)]
df = ada2021
#len(df['date'])
average = df['max5']-df['min5']
avg = average.mean()
avg
df
#point = df.loc[(df['max5'].diff()>0)]
#pointm = df.loc[(df['min5'].diff()>0)]

df['support5']= df.min5.rolling(5,5).min()
df['support10']= df.min10.rolling(10,10).min()
df['resist5']= df.max5.rolling(5,5).max()
df['resist10']= df.max10.rolling(10,10).max()

## try 
#df['support5']= df.min5

#df['RSI'] = RSI(df['close'])
#df['STRSI4']=STRSIA(df['close'],56,12,12)
#df['STRSI8']=STRSIA(df['close'],112,24,24)


#point = df[(df['max5'] > df['max5'].shift(1)) &  (df['max5'].shift(1) > df['max5'].shift(2))]
#pointm = df[(df['min5'] < df['min5'].shift(1)) &  (df['min5'].shift(1) > df['min5'].shift(2))]
#point
#ada2021
#pointm

#for row in range(0,len(pointm)):
#    print(pointm['support5'])
df

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,...,STRSIK8,STRSID8,max5,max10,min5,min10,support5,support10,resist5,resist10
0,2021-10-22 12:00:00,14.969,14.986,14.840,14.977,247200.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-10-22 13:00:00,14.978,14.978,14.639,14.669,366619.3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-10-22 14:00:00,14.670,14.670,14.203,14.323,1165891.3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-10-22 15:00:00,14.322,14.385,14.107,14.230,629394.1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-10-22 16:00:00,14.229,14.305,14.036,14.246,609145.0,NaN,NaN,NaN,NaN,...,NaN,NaN,14.986,NaN,14.036,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,2021-10-30 19:00:00,13.761,13.765,13.686,13.725,120067.7,56.798462,51.351847,48.381782,0.319816,...,NaN,NaN,13.779,14.191,13.494,13.414,13.414,13.144,14.191,14.191
200,2021-10-30 20:00:00,13.725,13.729,13.598,13.622,114787.9,52.429232,50.356345,47.888684,0.000000,...,NaN,NaN,13.779,14.191,13.546,13.414,13.414,13.144,14.054,14.191
201,2021-10-30 21:00:00,13.619,13.647,13.575,13.633,111160.1,52.846412,50.460773,47.945853,0.020163,...,NaN,NaN,13.779,14.054,13.575,13.414,13.414,13.144,14.054,14.191
202,2021-10-30 22:00:00,13.632,13.656,13.307,13.542,289927.1,49.016746,49.582250,47.510781,0.000000,...,NaN,NaN,13.779,14.054,13.307,13.307,13.307,13.179,13.869,14.191


In [43]:
buypoints = df[(df['STRSI'] < 0.01) &  (df['STRSI4'] < 0.01) & (df['STRSI8'] < 0.01) & (df['close'] <  df['support10'].shift(1)) & (df['RSI']<30)]
#buypoints = df[(df['STRSI'] < 0.01) &  (df['STRSI4'] < 0.01) & (df['STRSI8'] < 0.01)   & (df['RSI']<30)]

sellpoints = df[(df['STRSI'] == 1) &  (df['STRSI4'] == 1) & (df['STRSI8'] == 1) & (df['RSI']>90)]

#buypoints = df.loc[(df['STRSI'] <= 0.01) &  (df['STRSI4'] <= 0.01) & (df['STRSI8'] <= 0.01)]

sellpoints
buypoints

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,...,STRSIK8,STRSID8,max5,max10,min5,min10,support5,support10,resist5,resist10


In [44]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px



#Declare figure


fig = go.Figure()

fig = make_subplots(rows=8, cols=1,subplot_titles=("Price","gap","empty", "RSI 1 4 8", "Stoch 1", "Stoch 4","Stoch 8","Stoch 1 4 8"))
    

    
# buy points     
fig.add_trace(
    go.Scatter(x=buypoints.date, y=buypoints['close'],name="buy", mode = "markers+text"), row=1, col=1 
)    

# sell points     
fig.add_trace(
    go.Scatter(x=sellpoints.date, y=sellpoints['close'],name="sell", mode = "markers"), row=1, col=1 
) 
    
# buy points     
fig.add_trace(
    go.Scatter(x=buypoints.date, y=buypoints['resist10'],
        name="resist10",
        text="resist10",
        textposition="top center"),
    row=1, col=1
)  
# resistance 10 


fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI'],
        name="RSI",
        text="RSI",
        textposition="top center"),
    row=4, col=1
)

# support 10
fig.add_trace(
    go.Scatter(x=buypoints.date, y=buypoints['min10'],
        name="support10",
        text="support1010",
        textposition="top center"),
    row=1, col=1
)  

fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI4'],name="RSI4"),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI8'],name="RSI8"),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI'],name="STRSI"),
    row=5, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK'],name="STRSIK"),
    row=5, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID'],name="STRSID"),
    row=5, col=1
)
    
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI4'],name="STRSI4"),
    row=6, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK4'],name="STRSIK4"),
    row=6, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID4'],name="STRSID4"),
    row=6, col=1
)
    
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI8'],name="STRSI8"),
    row=7, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK8'],name="STRSIK8"),
    row=7, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID8'],name="STRSIK8"),
    row=7, col=1
)


fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI8'],name="STRSI8"),
    row=8, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI4'],name="STRSI4"),
    row=8, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI'],name="STRSI"),
    row=8, col=1
)


#add Traces

fig.add_trace(go.Candlestick(x=df.date,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'], name = 'market data'),row=1,col=1)

#fig.add_trace(go.Scatter(x=df['date'], y=df['max10']))

#fig.add_trace(go.Scatter(x=df['date'], y=df['min10']))

#fig.add_trace(go.Scatter(x=point['date'], y=point['resist5']))

#fig.add_trace(go.Scatter(x=pointm['date'], y=pointm['support5']))


#fig.add_shape(type="line",
#    x0=pointm['date'], y0=pointm['support5'], x1=max(pointm['date']), y1=pointm['support5'],
#    line=dict(
#        color="LightSeaGreen",
#        width=4,
#        dash="dashdot",
#    ))
fig.update_shapes(dict(xref='x', yref='y'))

#for index in range(0,len(pointm)):
    #fig.add_hline(y=pointm['support5'])

#fig.add_trace(go.Scatter(x=point['date'], y=point['resist10']))

#fig.add_trace(go.Scatter(x=pointm['date'], y=pointm['support10']))
#for i in range(len(df['date'])):
    #fig.add_hline(y=df[i]['min10'], line_dash="dot",
                  #annotation_text="support", 
                  #annotation_position="bottom right")

#fig.add_hline(y=df['max10'], line_dash="dot",
#              annotation_text="resitance", 
#              annotation_position="bottom right")



fig.update_layout(height=3000, width=800, title_text="top sub plots")
# X and Y Axis Update
fig.update_xaxes(
    title = 'Date',rangeslider_visible=False
)

#fig.update_yaxes(
#    title = 'Share Price (USD)'
#)

#Show
fig.show()